In [14]:
!pip3 install pyarrow

In [15]:
import os
from pathlib import Path
import geopandas as c
from pyspark.sql import SparkSession
import pandas
from pyspark.sql.types import IntegerType
from pyspark.sql import functions as func

In [16]:
raw_file_path = "/Users/kzmain/LSDE/data/*/raw/*/*.gz.parquet"

w_lon = -74.2463  # left bound
e_lon = -73.7141  # right bound
n_lat = 40.9166  # up bound
s_lat = 40.4767  # down bound

In [17]:
drop_lat = "dropoff_latitude"
drop_lon = "dropoff_longitude"
pick_lat = "pickup_latitude"
pick_lon = "pickup_longitude"
lat = "latitude"
lon = "longitude"

In [18]:
spark = SparkSession.builder.getOrCreate()

In [19]:
dfs = spark.read.parquet(raw_file_path)

In [20]:
def round_coordinate(_in_df):
    return _in_df\
        .withColumn(pick_lon, func.round(_in_df[pick_lon], 4)) \
        .withColumn(drop_lon, func.round(_in_df[drop_lon], 4)) \
        .withColumn(pick_lat, func.round(_in_df[pick_lat], 4)) \
        .withColumn(drop_lat, func.round(_in_df[drop_lat], 4))

In [21]:
def filter_by_bound(_in_df):
    return _in_df.filter(
        (e_lon >= func.col(pick_lon)) &
        (w_lon <= func.col(pick_lon)) &
        (e_lon >= func.col(drop_lon)) &
        (w_lon <= func.col(drop_lon)) &
        (n_lat >= func.col(pick_lat)) &
        (s_lat <= func.col(pick_lat)) &
        (n_lat >= func.col(drop_lat)) &
        (s_lat <= func.col(drop_lat))
    )

In [22]:
dfs = round_coordinate(dfs)
dfs = filter_by_bound(dfs)

In [23]:
pick = dfs.select([pick_lon, pick_lat]).withColumnRenamed(pick_lon, lon).withColumnRenamed(pick_lat, lat)
drop = dfs.select([drop_lon, drop_lat]).withColumnRenamed(drop_lon, lon).withColumnRenamed(drop_lat, lat)

both = pick.union(drop).drop_duplicates()

In [24]:
both.count()

KeyboardInterrupt: 

In [ ]:
# pick.count()3188093
# drop.count()4410010
# rslt.count()7598103
# rslt.count()5361090

In [ ]:
both.write.mode("overwrite").option("compression", "gzip").parquet("./node.parquet")